# Notebook to create a new C0free for the random library

In [2]:
'''
Imports
'''
proj_dir = '/Users/prabh/200 RESEARCH/Cluster Expansion Project'

# gives the notebook access to the cluster_expansion personal library
from sys import path
path.append(proj_dir)
import cluster_expansion as ce
from cluster_expansion import os, pd, np, plt, sns, tf, Markdown, display, train_test_split

/Users/prabh/200 RESEARCH/Cluster Expansion Project/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
model = tf.keras.models.load_model('../../CNN_models/RC_invariant/Symmetric/S_C0freeAvg.keras')

In [4]:
randomSeqs = pd.read_csv('Tiling_adapterfree.dat', sep = '\t')['Sequence']
randomSeqsRC = ce.SeqArr2RevCompArr(randomSeqs)

display(randomSeqs)

0        GTGATCTTCTCTTGCTCATTGCATTTGGAACTGAGACAGAAGAATG...
1        TCTCTTGCTCATTGCATTTGGAACTGAGACAGAAGAATGACGAGTT...
2        CTCATTGCATTTGGAACTGAGACAGAAGAATGACGAGTTACCACCC...
3        CATTTGGAACTGAGACAGAAGAATGACGAGTTACCACCCTCCTAGC...
4        AACTGAGACAGAAGAATGACGAGTTACCACCCTCCTAGCATCTGGC...
                               ...                        
81796    ATCATCTACATGAAAAAAACTCCCGCAATTTCTTATAGAATACGTT...
81797    ACATGAAAAAAACTCCCGCAATTTCTTATAGAATACGTTGAAAATT...
81798    AAAAACTCCCGCAATTTCTTATAGAATACGTTGAAAATTAAATGTA...
81799    CCCGCAATTTCTTATAGAATACGTTGAAAATTAAATGTACGCGCCA...
81800    TTTCTTATAGAATACGTTGAAAATTAAATGTACGCGCCAAGATAAG...
Name: Sequence, Length: 81801, dtype: object

In [5]:
randomOnehot = ce.Data2Onehot(randomSeqs).reshape(-1, 200, 1)
randomOnehotRC = ce.Data2Onehot(randomSeqsRC).reshape(-1, 200, 1)

Encoded all sequences to onehot
Encoded all sequences to onehot


In [6]:
randomC0s = model.predict(randomOnehot).reshape(-1,)
randomC0sRC = model.predict(randomOnehotRC).reshape(-1,)

print("Correlation between randomC0s and randomC0sRC: %.4f", np.corrcoef(randomC0s, randomC0sRC)[0, 1])

2557/2557 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step
2557/2557 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step
Correlation between randomC0s and randomC0sRC: %.4f 0.9925511807287952


In [7]:
display(pd.Series(randomC0s))

0       -0.579442
1       -0.568332
2       -0.434555
3       -0.126269
4       -0.052728
           ...   
81796    0.088059
81797    0.105487
81798    0.019940
81799   -0.207853
81800   -0.269665
Length: 81801, dtype: float32

In [8]:
originalC0s = pd.read_csv('Tiling_adapterfree.dat', sep = '\t')['C0free'].values

In [9]:
correlation = np.corrcoef(randomC0s, originalC0s)[0, 1]
print(f"Correlation between randomC0s and originalC0s: {correlation:.4f}")

Correlation between randomC0s and originalC0s: 0.9607


In [10]:
# organize a dataframe of sequences and their predicted C0 values
assert len(randomSeqs) == len(randomC0s)
df_random = pd.DataFrame({'Sequence': randomSeqs.values, 'randomC0': randomC0s})
display(df_random.head())

,Sequence,randomC0
0,GTGATCTTCTCTTGCTCATTGCATTTGGAACTGAGACAGAAGAATG...,-0.579442
1,TCTCTTGCTCATTGCATTTGGAACTGAGACAGAAGAATGACGAGTT...,-0.568332
2,CTCATTGCATTTGGAACTGAGACAGAAGAATGACGAGTTACCACCC...,-0.434555
3,CATTTGGAACTGAGACAGAAGAATGACGAGTTACCACCCTCCTAGC...,-0.126269
4,AACTGAGACAGAAGAATGACGAGTTACCACCCTCCTAGCATCTGGC...,-0.052728


In [ ]:
# save df_random to a tab-separated .dat file
save = False
if save:
  df_random.to_csv('Tiling_S_C0freeAvg.dat', sep='\t', index=False, header = ['Sequence', 'C0free'], float_format='%.7f')